# AI Rapper

### _Generate rap lyrics with different AI and NLP techniques_

We pull rap lyrics off of the Genius API, then feed them through a RNN, eventually asking the network to produce new text from a seed.

In [2]:
%matplotlib inline

import numpy as np
import random
import sys
import os
import datetime
import io
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import json

import keras
from keras.callbacks import TensorBoard
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


### Download the Lyrics Data 

In [81]:
import lyricsgenius as genius
geniusCreds = "Lw6NjXtbU7NndUFHRCcOX9FdLhPzVokLIt9c4LWzsTxM10wF7EICGtWSSso8Ohsq"
artist_name = "DaBaby"

api = genius.Genius(geniusCreds)
artist = api.search_artist(artist_name, max_songs=20)
artist.save_lyrics()

Searching for songs by DaBaby...

Song 1: "Suge"
Song 2: "INTRO"
Song 3: "BOP"
Song 4: "Next Song"
Song 5: "Goin Baby"
Song 6: "VIBEZ"
Song 7: "Baby Sitter"
Song 8: "TOES"
Song 9: "21"
Song 10: "Today (Intro)"
Song 11: "Walker Texas Ranger"
Song 12: "Blank Blank"
Song 13: "OFF THE RIP"
Song 14: "Pull Up Music"
Song 15: "XXL"
Song 16: "4x"
Song 17: "GOSPEL"
Song 18: "POP STAR"
Song 19: "RAW SHIT"
Song 20: "Pony"

Reached user-specified song limit (20).
Done. Found 20 songs.
Wrote `Lyrics_DaBaby.json`


### Load the Data and Produce Basic Baseline Model

In [3]:
# Reading the json as a dict
text = ""
for filename in os.listdir('./data/'):
    with open('./data/' + filename) as json_data:
        data = json.load(json_data)

        for song in data['songs']:
            text += song['lyrics']

In [6]:
from collections import Counter

cnt = Counter()

for char in text:
    cnt[char] += 1

most_com = cnt.most_common()[0]

# if we were to run a baseline model, predicting every character as a space
correct = 0
for char in text:
    if char == most_com[0]:
        correct += 1
        
# print the accuracy
print(f'Baseline Model Accuracy: {correct/len(text)}')

Baseline Model Accuracy: 0.1700664496363421


### RNN Construction & Training

In [11]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #

def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(200):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=75,
          callbacks=[])

Epoch 1/75
25549/25549 [==============================] - 24s 927us/step - loss: 2.4127 - categorical_crossentropy: 2.4127 - accuracy: 0.3415
Epoch 2/75
25549/25549 [==============================] - 24s 928us/step - loss: 1.7506 - categorical_crossentropy: 1.7506 - accuracy: 0.4839
Epoch 3/75
25549/25549 [==============================] - 23s 914us/step - loss: 1.5949 - categorical_crossentropy: 1.5949 - accuracy: 0.5229
Epoch 4/75
25549/25549 [==============================] - 23s 887us/step - loss: 1.5253 - categorical_crossentropy: 1.5253 - accuracy: 0.5409
Epoch 5/75
25549/25549 [==============================] - 23s 888us/step - loss: 1.4826 - categorical_crossentropy: 1.4826 - accuracy: 0.5526
Epoch 6/75
25549/25549 [==============================] - 23s 884us/step - loss: 1.4547 - categorical_crossentropy: 1.4547 - accuracy: 0.5603
Epoch 7/75
25549/25549 [==============================] - 22s 876us/step - loss: 1.4332 - categorical_crossentropy: 1.4332 - accuracy: 0.5665
Epoch 

### Rap Lyric Generation

In [13]:
print()
print('... Generating lyrics')
diversity = .6
print('... With diversity value:', diversity)

generated = ''
sentence = 'Like a shepherd having sex with a sheep, fuck what you heard'[:40]
generated += sentence
print('... Generating with seed: "' + sentence + '"')
print()
sys.stdout.write(generated)

for i in range(5000):
    x_pred = np.zeros((1, seqlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.
    preds = model.predict(x_pred, verbose=0)
    next_index = sample(preds[0, -1], diversity)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()


... Generating lyrics
... With diversity value: 0.6
... Generating with seed: "Like a shepherd having sex with a sheep,"

Like a shepherd having sex with a sheep, pussy is love
Way too many fly with your mind, where the case I ain't nothin'
They like when I was too much for my shit
I was too bushed, you got a Pati, and the mall you with the star
When you could talk, you gon' be all day with the ones
I love it was pretendays my family shit
I want you what you know where I chelf
I'm just everybody believe it I stay toure on the club
I ain't take a polis that I was learn and these nigga one bread for sure
I'm talkin' about her doing mornin', this man with you
And I never liked the club up (Yeah)

[Verse 2: Travis Scott]
I ain't lookin' for me way down
For the schemin' shot as he askin' with you

[Verse 2]
I hope you know what's that she get it, I got to show me (Keep gon' go back)
I see my waves all in
All the way I want my team (row)
Get it, get it, count it up, count it, count it
I wan

### n-Gram Model

In [22]:
import re
NUM_OF_ITS = 10000

regex = r"[\w']+|[\n]"
document = []

words = re.findall(regex, text)

print(words[:20])

['Produced', 'by', 'Phonix', 'Beats', 'and', 'J', 'Cole', '\n', '\n', 'Verse', '1', '\n', 'First', 'things', 'first', 'rest', 'in', 'peace', 'Uncle', 'Phil']


In [13]:
from collections import defaultdict
from tqdm import tqdm

transitions = defaultdict(list)
for prev, current in zip(words, words[1:]):
    transitions[prev].append(current)

def generate_using_bigrams():
    current = '\n'
    result = []
    count = 0
    while True:
        next_word_candidates = transitions[current]
        current = random.choice(next_word_candidates)
        result.append(current)
        if count == NUM_OF_ITS: return " ".join(result)
        count += 1

res = generate_using_bigrams()
print(res)

For sure you is the best show 
 
 Mamacita 'cita 
 My tailor see 
 Suck a TEC Rack 
 
 Don't let it at me 
 I I want to Produced by the flow that fuck the song is complicated 
 Never let niggas I put on let's get your summers in ages 
 Fill PJ's up 50 tell I move out to a groupie 
 
 Verse 4 Kendrick Lamar 
 
 Wonder if she say no mind or swim moves 
 She told me down for a virgin 
 Nigga play Mr Perfect 
 Keep on my


In [23]:
trigram_transitions = defaultdict(list)
starts = []

for prev, current, nxt in zip(words, words[1:], words[2:]):
    if prev == '\n':
        starts.append(current)
    trigram_transitions[(prev, current)].append(nxt)
    
def generate_using_trigrams():
    current = random.choice(starts)
    prev = '\n'
    result = [current]
    count = 0
    
    while True:
        next_word_candidates = trigram_transitions[(prev, current)]
        next_word = random.choice(next_word_candidates)
        
        prev, current = current, next_word
        result.append(current)
        if count == NUM_OF_ITS: return ' '.join(result)
        count += 1
        
print(generate_using_trigrams())
        

If I said I'm trapped in my grip she holding tight to a nigga palms sweatin' 
 Baby mama cover Forbes got these new rappers nervous prom night nigga 
 
 That can only mean one thing 
 
 Chorus Jhené Aiko 
 Feels like the light don't hit my Ts with Febreze 
 Jordan pull up they need me Need me a show and get high 'cause the world like Where do we go yeah 
 All through the late night And then I see some shit with that thang that I can't decide they keep switchin' majors 
 Being indecisive makes me nervous 
 Fuck the rest in peace to Screw tonight we take it slowly 
 Put the key to the studio every night for you always 
 
 Outro 
 Money trees is the symbol for the league 
 And I know I'm on the next game played I might as well overheat 
 Too busy thinking what if yeah 
 
 Outro Kendrick Lamar 
 This is what you if you've felt too insecure 
 Hold up wait more to life than sleeping in my life 
 The furthest thing from heaven 
 No conspiracy my fate 
 Gonna be your knight in shining Armor Al